In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
#from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from time import time

In [40]:
#path = '/Users/Vishy/Files/AVDatahack/LOM/Data/' 
path = '/Users/804357/Desktop/MyFiles/Learn/LOM/Data/'
train = pd.read_csv(path+'train.csv')
train = train.head(100000)
test = pd.read_csv(path+'test.csv')
test = test.head(8000)
camp = pd.read_csv(path+'campaign_data.csv')
print(train.shape, test.shape, camp.shape)

(100000, 6) (8000, 4) (52, 9)


In [41]:
train = pd.merge(train, camp, on='campaign_id')
test = pd.merge(test, camp, on='campaign_id')
print(train.shape, test.shape)

(100000, 14) (8000, 12)


In [42]:
train.head()

,id,user_id,campaign_id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,42_14051,14051,42,01-09-2017 19:55,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
1,42_177808,177808,42,01-09-2017 20:13,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
2,42_133077,133077,42,01-09-2017 20:11,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
3,42_118677,118677,42,01-09-2017 20:15,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
4,42_25809,25809,42,01-09-2017 19:49,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...


In [43]:
test.head()

,id,campaign_id,user_id,send_date,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,63_122715,63,122715,01-02-2018 22:35,Newsletter,68,64,15,5,\r\nFebruary 2018 Newsletter\r\n \r\nDear AVia...,"AVbytes, Ultimate 2018 learning path and aweso...",http://r.newsletters.analyticsvidhya.com/7whsu...
1,63_124394,63,124394,01-02-2018 22:47,Newsletter,68,64,15,5,\r\nFebruary 2018 Newsletter\r\n \r\nDear AVia...,"AVbytes, Ultimate 2018 learning path and aweso...",http://r.newsletters.analyticsvidhya.com/7whsu...
2,63_95168,63,95168,01-02-2018 22:44,Newsletter,68,64,15,5,\r\nFebruary 2018 Newsletter\r\n \r\nDear AVia...,"AVbytes, Ultimate 2018 learning path and aweso...",http://r.newsletters.analyticsvidhya.com/7whsu...
3,63_31556,63,31556,01-02-2018 23:09,Newsletter,68,64,15,5,\r\nFebruary 2018 Newsletter\r\n \r\nDear AVia...,"AVbytes, Ultimate 2018 learning path and aweso...",http://r.newsletters.analyticsvidhya.com/7whsu...
4,63_138377,63,138377,01-02-2018 22:48,Newsletter,68,64,15,5,\r\nFebruary 2018 Newsletter\r\n \r\nDear AVia...,"AVbytes, Ultimate 2018 learning path and aweso...",http://r.newsletters.analyticsvidhya.com/7whsu...


In [44]:
train.columns
# test.columns

Index(['id', 'user_id', 'campaign_id', 'send_date', 'is_open', 'is_click',
       'communication_type', 'total_links', 'no_of_internal_links',
       'no_of_images', 'no_of_sections', 'email_body', 'subject', 'email_url'],
      dtype='object')

In [45]:
# Processing date
train['send_date'] = pd.to_datetime(train['send_date'])
test['send_date'] = pd.to_datetime(test['send_date'])

In [46]:
# Feature Engineering to Create New variables for Day of Week and Day part
#train
train['Weekday'] = (train['send_date'].dt.dayofweek).astype(int)
train['Dayhour'] = (train.send_date.apply(lambda x: x.hour)).astype(int)
train['Month'] = pd.DatetimeIndex(train['send_date']).month

#test
test['Weekday'] = (test['send_date'].dt.dayofweek).astype(int)
test['Dayhour'] = (test.send_date.apply(lambda x: x.hour)).astype(int)
test['Month'] = pd.DatetimeIndex(test['send_date']).month

In [47]:
# To remove Ordinality in both Weekday and DayHour variables convert into Categories
#train
train['Weekday'] = train['Weekday'].map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thr',4: 'Fri', 5: 'Sat', 6: 'Sun'} ).astype(str)
train['Dayhour'] = train['Dayhour'].map({0:'AM1', 1:'AM1', 2:'AM1', 3:'AM2', 4:'AM2', 5:'AM2', 6:'AM3', 7:'AM3', 8:'AM3',
                                         9:'AM4', 10:'AM4', 11:'AM4', 12:'PM1', 13:'PM1', 14:'PM1', 15:'PM2', 16:'PM2',
                                         17:'PM2', 18:'PM3', 19:'PM3', 20:'PM3', 21:'PM4', 22:'PM4', 23:'PM4'}).astype(str)
train['Month'] = train['Weekday'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 
                                        10:'Oct', 11:'Nov', 12:'Dec'} ).astype(str)

#test
test['Weekday'] = test['Weekday'].map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thr', 4:'Fri', 5:'Sat', 6:'Sun'}).astype(str)
test['Dayhour'] = test['Dayhour'].map({0:'AM1', 1:'AM1', 2:'AM1', 3:'AM2', 4:'AM2', 5:'AM2', 6:'AM3', 7:'AM3', 8:'AM3',
                                       9:'AM4', 10:'AM4', 11:'AM4', 12:'PM1', 13:'PM1', 14:'PM1', 15:'PM2', 16:'PM2',
                                       17:'PM2', 18:'PM3', 19:'PM3', 20:'PM3', 21:'PM4', 22:'PM4', 23:'PM4'}).astype(str)
test['Month'] = test['Weekday'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 
                                        10:'Oct', 11:'Nov', 12:'Dec'}).astype(str)

In [48]:
# Create final day time variable
train['DayPeriod'] = (train['Month']+train['Weekday']+train['Dayhour']).astype(str)
test['DayPeriod'] = (test['Month']+test['Weekday']+train['Dayhour']).astype(str)
del train['send_date'],train['Weekday'],train['Dayhour'],test['send_date'],test['Weekday'],test['Dayhour'],train['Month'],test['Month']

In [49]:
# prep data for Base Model
y_train = train['is_click'].values
id_test = test['id'].values
id_train = train['id'].values

del train['id'],train['user_id'],train['is_open'],train['is_click'],train['email_body'],train['subject'],train['email_url']
del test['id'],test['user_id'],test['email_body'],test['subject'],test['email_url']

print(train.shape,test.shape)

(100000, 7) (8000, 7)


In [50]:
train_cats = pd.get_dummies(data=train, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
test_cats = pd.get_dummies(data=test, columns=['campaign_id','communication_type','DayPeriod'], drop_first=True)
print (train_cats.shape,test_cats.shape)

(100000, 54) (8000, 41)


In [51]:
train_cats.head()

,total_links,no_of_internal_links,no_of_images,no_of_sections,campaign_id_30,campaign_id_31,campaign_id_32,campaign_id_33,campaign_id_34,campaign_id_35,...,DayPeriod_nanSunPM4,DayPeriod_nanThrPM1,DayPeriod_nanThrPM2,DayPeriod_nanThrPM3,DayPeriod_nanThrPM4,DayPeriod_nanTueAM4,DayPeriod_nanTuePM3,DayPeriod_nanWedAM4,DayPeriod_nanWedPM1,DayPeriod_nanWedPM2
0,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,88,79,13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [53]:
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['random_state'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.01
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['random_state'] = 99

lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['random_state'] = 99

In [54]:
lgb_model = LGBMClassifier(**lgb_params)
lgb_model2 = LGBMClassifier(**lgb_params2)
lgb_model3 = LGBMClassifier(**lgb_params3)

In [55]:
log_model = LogisticRegression()
stack = Ensemble(n_splits=10, stacker = log_model,base_models = (lgb_model, lgb_model2, lgb_model3))
y_pred = stack.fit_predict(train_cats, y_train, test_cats)

Fit LGBMClassifier fold 1


C:\Users\804357\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\804357\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


ValueError: Number of features of the model must match the input. Model n_features_ is 54 and input n_features is 41 

In [ ]:
out = pd.DataFrame({'id': id_test, 'is_click': y_pred})
out.to_csv("LGBStack.csv", index=False)